In [9]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
import geopandas as gpd
import pandas as pd
import fiona
import matplotlib.pyplot as plt
import glob
import numpy as np
plt.rcParams["figure.figsize"] = [10, 10]

In [15]:
zipc = gpd.read_file('zipcodes/ZIP_CODE_040114.shp')
df = pd.read_csv('web/static/result_day.csv')
print zipc.crs
zipc = zipc.to_crs(epsg=4326)
print zipc.crs
# zipc.to_json('web/static/zipcodes.json', orient='records')

{u'lon_0': -74, u'datum': u'NAD83', u'y_0': 0, u'no_defs': True, u'proj': u'lcc', u'x_0': 300000, u'units': u'us-ft', u'lat_2': 41.03333333333333, u'lat_1': 40.66666666666666, u'lat_0': 40.16666666666666}
{'init': 'epsg:4326', 'no_defs': True}


In [16]:
zipdict = {}
def dec(x):
    z = x['ZIPCODE']
    if z not in zipdict:
        zipdict[int(z)] = x.__dict__
        r = {}
        for k in zipc.columns:
            r[k.lower()] = x[k]
            r['geometry'] = x['geometry'].centroid
        del r['url']
        del r['state']
        del r['county']
        del r['shape_area']
        del r['shape_len']
        del r['bldgzip']
        zipdict[int(z)] = r 
    
a = zipc.apply(lambda x: dec(x), axis=1)
zipdict.values()[0]['geometry'].__dict__

{'__geom__': 69500064, '__p__': None, '_is_empty': False, '_ndim': 2}

In [18]:
from datetime import date, timedelta
import datetime

def conv(x):
    x = int(x)
    d = datetime.datetime.fromtimestamp(x)
    return d


def extract(s, x):
    d1 = conv(x['startTime'])
    d2 = conv(x['endTime'])
    delta = d2 - d1         # timedelta
    ret = []
    for i in range(delta.days + 1):
        r = d1 + timedelta(days=i)
        r = r.strftime("%Y-%m-%d")
        ret.append(r)
    return len(ret) if s==0 else ret
    
df1 = df[:]
df1.reset_index()
df1['ndays'] = df1.apply(lambda x: extract(0,x), axis=1)
df1['days'] = df1.apply(lambda x: extract(1,x), axis=1)

/home/fhorta/miniconda2/envs/bigdata/lib/python2.7/site-packages/ipykernel/__main__.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/fhorta/miniconda2/envs/bigdata/lib/python2.7/site-packages/ipykernel/__main__.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [19]:
from shapely.geometry import mapping

def getloc(x, what):
    x = x[what]
    try:
        x = int(x)
        geom = zipdict[x]['geometry']
        feature = [{'type': 'Feature', 'properties': {}, 'geometry': mapping(geom)}]
        return feature
    except:
        return ''
    
df1['geom_complaint'] = df1.apply(lambda x: getloc(x, 'Zip_311'), axis=1)
df1 = df1[df1['geom_complaint']!='']
df1['geom_crime'] = df1.apply(lambda x: getloc(x, 'Zip_NYPD'), axis=1)
df1 = df1[df1['geom_crime']!='']

/home/fhorta/miniconda2/envs/bigdata/lib/python2.7/site-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [20]:
ziputm = {}
def dec(x):
    z = x['ZIPCODE']
    if z not in zipdict:
        ziputm[int(z)] = x.__dict__
        r = {}
        for k in zipc.columns:
            r[k.lower()] = x[k]
            r['geometry'] = x['geometry'].centroid
        del r['url']
        del r['state']
        del r['county']
        del r['shape_area']
        del r['shape_len']
        del r['bldgzip']
        ziputm[int(z)] = r 
    
temp = zipc.to_crs(epsg=26918)
a = temp.apply(lambda x: dec(x), axis=1)

def dist(x):
    i = ziputm[int(x['Zip_311'])]['geometry']
    j = ziputm[int(x['Zip_NYPD'])]['geometry']
    return i.distance(j)

df1['distance'] = df1.apply(lambda x: dist(x), axis=1 )

In [21]:
def g(x):
    s = x['Relationship']
    if s==1:
        return '311->Crime'
    if s==2:
        return 'Crime->311'
    else:
        return 'No causality'

df1['group'] = df.apply(lambda x: g(x), axis=1)

In [22]:
def g(x):
    s = x['Relationship']
    a1 = x['311']
    a2 = x['NYPD']
    
    if s==1:
        return '%s > %s' % (a1,a2)
    if s==2:
        return '%s > %s' % (a2,a1)
    else:
        return '%s || %s' % (a1,a2)

df1['name'] = df.apply(lambda x: g(x), axis=1)

In [23]:
def fmt(x,w):
    x = int(x[w])
    d = datetime.datetime.fromtimestamp(x)
    d = d.strftime("%Y-%m-%d")
    return d

df1.startTime = df.apply(lambda x: fmt(x,'startTime'), axis=1)
df1.endTime = df.apply(lambda x: fmt(x,'endTime'), axis=1)

In [53]:
cond = {}

def getid(x):
    s = x['Relationship']
    a1 = x['311']
    a2 = x['NYPD']
    z1 = x['Zip_311']
    z2 = x['Zip_NYPD']
    if s==1:
        return a1 + a2 
    if s==2:
        return a2 + a1
    else:
        return None

def g(x):
    i = getid(x)
    if not i: 
        return
    ndays = int(x['ndays'])
    if i not in cond:
        cond[i] = ndays
    else:
        cond[i] += ndays 
        
def score(x):
    i = getid(x)
    if not i or i not in cond:
        return 0
    else:
        return cond[i]

a = df1.apply(lambda x: g(x), axis=1)
max = np.array(cond.values()).max()
for k,c in cond.iteritems():
    cond[k] = -(float(float(c)/max*1.) - 1)
               
print np.array(cond.values()).max()
print np.array(cond.values()).min()
df1['score'] = df.apply(lambda x: score(x), axis=1)
cond.keys()[:2]

0.999935072181
-0.0


['Blocked DrivewayVEHICLE AND TRAFFIC LAWS', 'HARRASSMENT 2Illegal Parking']

In [54]:
df1.reset_index().to_json('web/static/result_day-pos2.json', orient='records')

In [14]:
# df1_light = df1.drop(['geom_crime', 'geom_complaint'], axis=1)
df1_light.to_json('web/static/result_day-pos2-light.csv')

NameError: name 'df1_light' is not defined

In [ ]:
df2 = pd.read_csv('web/static/streets.csv')

In [ ]:
df2.head()

In [ ]:
df1.group.head()